# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [9]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****hPcA
LANGSMITH_API_KEY=****a745
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [10]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [11]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [12]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [13]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [14]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
my_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    max_tokens=None,
    temperature=0 # Lower temperature for stable, factual SQL generation
)#new
agent = create_agent(
    #model="openai:gpt-5"
    model=my_llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [15]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (f3fab9e8-6f63-4fa5-aaae-7074f9f4b35b)
 Call ID: f3fab9e8-6f63-4fa5-aaae-7074f9f4b35b
  Args:
    query: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (f10ff81e-d9ec-4d8e-9797-4779afe5bcac)
 Call ID: f10ff81e-d9ec-4d8e-9797-4779afe5bcac
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


...

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (408c204f-57b5-4444-93eb-76daa6bf9b64)
 Call ID: 408c204f-57b5-4444-93eb-76daa6bf9b64
  Args:
    query: SELECT Total FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(5.94,)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': 'The total on your last invoice was 5.94.', 'extras': {'signature': 'Cv0DAdHtim+m6OTi3W90Di1eZdcu17Chx/B4MdyIH8OOmKs/m88fdtBSCBL1+yUZvhm6keABf/wYGjA9+ip5Rc5JMZqAANbYRIGecBWALaq0IUhACdBg+73fPgswYlSxb6P9/zdSX0SwX9fdIRUFoN/ywwh2IrGuQW7zsUXGI+25Abzu9UFWNdi2NsHbASUE04aw7LPx7P/2OV2750tgSaeFyMxrat72tAmk21NWCmiDUxW4RMKrujOMY3WPp8sUfbNcwhB6NRb9ECepF1BADXHdWV98DfPEwRDO9PnLlhU+bvWWC5wJocMj0e9VzqLO+lqoPr9wA1+zCxEkAu7CTNmpvuidqV4TgkMojnzQwOTtWC+BMJOszBLAeL4InIgeW7H0ays9nPJ8NxhHmYBbs7MQEkGmJVMdSQczbGDFHxedKJCxm1RpamtRQc79kB7iWUUpk2SQ2pNL44XItAGC9LIgSg4KsAHsYtI6WNVus5jBqe+VrUCbvuoBJfy0L+FwHxx/nIw+ItPCQc7sx9bLXGxuUhQ+lFOKJvi4zx0IjZLUPdyfZV4XzkmIhkWgCjzCqa0WNug3yzeC09FFUQS5CnsmzvOqY1Li+D3IPbrhKht2vww0/TKzYmXOgPBJZo+G0hxsKsxuGeqYYdHUF7ipnrmzeSBsXND5i3VVr0QDanY='}}]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [16]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (f969bb7d-874d-44a6-ae27-052e50390625)
 Call ID: f969bb7d-874d-44a6-ae27-052e50390625
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (ff9dc2ae-0a2b-4cab-8744-f0638a33a5df)
 Call ID: ff9dc2ae-0a2b-4cab-8744-f0638a33a5df
  Args:
    query: SELECT name FROM pragma_table_info('Track');
================================= Tool Message =================================
Name: execute_sql

[('TrackId',), ('Name',), ('AlbumId',), ('MediaTypeId',), ('GenreId',), ('Composer',), ('Milliseconds',), ('Bytes',), ('UnitPrice',)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (f28f6fa1-4929-438f-989a-48a5010ff5be)
 Call ID: f28f6fa1-4929-438f-989a-48a5010ff5be
  Args:
    query: SELECT Name FROM Track LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('For Those About To Rock (We Salute You)',), ('Balls to the Wall',), ('Fast As a Shark',), ('Restless and Wild',), ('Princess of the Dawn',)]
================================== Ai Message ==================================

The titles are:
* For Those About To Rock (We Salute You)
* Balls to the Wall
* Fast As a Shark
* Restless and Wild
* Princess of the Dawn


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## Add memory

In [19]:
from langgraph.checkpoint.memory import InMemorySaver # for short term memory

In [20]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    #model="openai:gpt-5"
    model=my_llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [21]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (c3784478-2251-462d-b43f-6d01158194c8)
 Call ID: c3784478-2251-462d-b43f-6d01158194c8
  Args:
    query: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (3961b31c-1320-4b6c-9b70-3989c79728c2)
 Call ID: 3961b31c-1320-4b6c-9b70-3989c79728c2
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


...

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (4ce8715a-7b78-4bcd-84ae-8e75720a56db)
 Call ID: 4ce8715a-7b78-4bcd-84ae-8e75720a56db
  Args:
    query: SELECT Total FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(5.94,)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': 'The total on your last invoice was 5.94.', 'extras': {'signature': 'Cv0DAdHtim/Jw1S22nCx3ah5o6j/N7zbW7bAzUorSfFdqp9up9Ad8JF9OfcMePzg9uHVQKEtJI/iEhrZFFBGIGPJlOJUSdAAYM2Rd8Q71bmPiwzm/XaeQAaMjw3NPS3jGQEHKk1rWQgyrpUNDPdME8XoYdQtGQUtYZjzd5T3Sa5Dz8SXdxGWfXBWTSgC7Mv2wjHEwvYJar8QrlUIE9lDuA4MHFy+xCWakNqFctauRXMUiyrSL7zUzBBu4wmK1WV4qj43O9A3PHgHAb8HKeUqkPWfJOMPmNU8c5D1F4p06Q1T45OTRYdaMcOuNDr3bRhkmC1do8Ujo772QWDH0oAxFV3bHBzbuvru3FiILvD+P9gjDc5n2rNwFv2az6DQUZuzBEHMlguoQOHjn7TVWs41CWWy9lZcdEEKPEcjLSAhxNtQktResDBR8/dQ8dCjVP6ZUcb84SFvwjKUtUkPaszG9eqXkxpHjw/CvH6St6BOcZFbhrHm+dEMUT08gvgcFEKl9KIo/actztuuZiiSXooF+i+bH/I4nqFSCwyeS8+aGgTfLpvu1owGOCsXg1L3uVjJvFNQVBBMXWXjMAGzSpLsfzkQhX6Sq2+fkTDcciXTWrDx0K3VpshGG5y0SwAeISzLmFcJQwSGAe6LBPLwrqNGy2URpLEj2HXx7qMepq9T+VY='}}]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [22]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (0cc8c79d-305f-4199-bdf0-94a73208eb76)
 Call ID: 0cc8c79d-305f-4199-bdf0-94a73208eb76
  Args:
    query: SELECT InvoiceId FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(374,)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (fde5bc96-084e-4565-8258-3eee6ba88173)
 Call ID: fde5bc96-084e-4565-8258-3eee6ba88173
  Args:
    query: SELECT T.Name FROM Track AS T JOIN InvoiceLine AS IL ON T.TrackId = IL.TrackId WHERE IL.InvoiceId = 374
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',), ('So What',)]
================================== Ai Message ==================================

The titles on your last invoice were: 'Holier Than Thou', 'Through The Never', 'My Friend Of Misery', 'The Wait', 'Blitzkrieg', and 'So What'.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## Try your own queries
Now that there is memory, check the agents recall!

In [23]:
question = "Sort the titles in order of money spent"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Sort the titles in order of money spent


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (925eb383-5bca-48e2-af20-ea0cfe04c67d)
 Call ID: 925eb383-5bca-48e2-af20-ea0cfe04c67d
  Args:
    query: SELECT T.Name, IL.UnitPrice FROM Track AS T JOIN InvoiceLine AS IL ON T.TrackId = IL.TrackId WHERE IL.InvoiceId = 374 ORDER BY IL.UnitPrice DESC
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou', 0.99), ('Through The Never', 0.99), ('My Friend Of Misery', 0.99), ('The Wait', 0.99), ('Blitzkrieg', 0.99), ('So What', 0.99)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': "All the titles on your last invoice cost the same amount (0.99 each), so sorting them by money spent will not change their order. The titles are: 'Holier Than Thou', 'Through The Never', 'My Friend Of Misery', 'The Wait', 'Blitzkrieg', and 'So What'.", 'extras': {'signature': 'CqMDAdHtim+4mD4eei9eZQjb4oDrrHr3oYDh9fyHeoBmMWjxq68vtiG2UED0Kn+TmhkDRZ7VN5kgQcAH7gzTL5UhPR2gdgtpv1SJCs4PnCt5ZWQ/S9BC3PrggKA67yNSw8NZqbEf9WXKqSGoYGfSkn/jJ1uM0Bu7THU7Eo3RxLcjgcSyS0ttwtkOBSf7zDCCX1QfY79e3e3gr2VzFOsqcVfClVlMjGs6Z+Yp8yY58+cvW/FWIWm/Rz9OJSR6wFH7dTOMbhIsUNul7ucmsV7+K7LbJClLjW1TARW/qj1wBSCg1tCm4WzO5DFJ/3mNK7vxwEOaOUfzEBVHjWECAz/JaXJdUfxkHVbxRQHvsQ8kBh+Mv9bEjbsUHvZvUKGeRN+iaCqjkVMTbik3P5sU9trZoCQ2EXi4qZLQ9Dx8IFGO5SLa9tHgsqstQsO0/ma20cyIjs49j/7413s20ckMbL+WmN+ZIX2ivNWH0AXZzDsZib4+GoxHp73a3tJl3GdQajobCESY5V87jMfvwgXSe4yhZt2JcXvZBTmD7lQ/7tCJadI2Dtt1d7Q='}}]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
